
# TP1 : Intent recognition

## EX0 1 :

In [14]:
# Importing dependencies 
import pandas as pd 
import numpy as np
import os

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import xgboost as xgb

from sklearn.metrics import accuracy_score


## Incorporating the Kismet data into a dataset

In [15]:
# Define the directory path where the files are located
directory_path = 'Kismet_data_intent'

# Create an empty list to store the dataframes
data_set = pd.DataFrame(columns=['speaker', 'nbr', 'classe', 'time', 'f0', 'en'])
i = 0
# Loop through each file in the directory
for filename in os.listdir(directory_path):

    speaker = filename[0:2]
    nbr = filename[2:6]
    classe = filename[6:8]


    if not (np.any(data_set.loc[data_set['nbr'] == nbr ,'classe'] == classe) and np.any(data_set.loc[data_set['nbr'] == nbr ,'speaker'] == speaker)) :


        filename = filename[:-3]
        path_en = 'Kismet_data_intent/'+ filename +'.en'
        path_f0 = 'Kismet_data_intent/'+ filename +'.f0'

        #print(path_en)
        data_en = np.genfromtxt(path_en, delimiter=' ')
        data_f0 = np.genfromtxt(path_f0, delimiter=' ')

        time = data_en[:,0]

        data_en = data_en[:,1]
        data_f0 = data_f0[:,1]
        if data_set.index.max() >= 0 :
            
            data_set.loc[data_set.index.max() + 1] = [speaker, nbr, classe, time, data_f0, data_en] 
        else :
            data_set.loc[0] = [speaker, nbr, classe, time, data_f0, data_en] 

    else :
        if filename.endswith('.en'):
            print(filename)

data_set

,speaker,nbr,classe,time,f0,en
0,cy,0001,at,"[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 42.0, 41.0, 43.0, 41.0, 39.0, 39.0,..."
1,cy,0002,at,"[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 40.0, 40.0, 41.0, 40.0, 39.0, 41.0,..."
2,cy,0005,at,"[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 42.0, 41.0, 43.0, 41.0, 39.0, 39.0,..."
3,cy,0006,at,"[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 40.0, 40.0, 41.0, 40.0, 39.0, 41.0,..."
4,cy,0007,pw,"[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 43.0, 41.0, 40.0, 38.0, 39.0, 40.0, 41.0..."
...,...,...,...,...,...,...
534,li,1446,at,"[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 40.0, 40.0, 40.0, 41.0, 41.0, 39.0,..."
535,li,1447,at,"[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 40.0, 42.0, 46.0, 45.0, 43.0, 43.0,..."
536,li,1448,at,"[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 40.0, 40.0, 40.0, 39.0, 39.0, 40.0,..."
537,li,1449,at,"[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 40.0, 41.0, 38.0, 38.0, 39.0, 39.0,..."


## Compiling statistical information

In [16]:
def stats(arr) :
    # mean
    mean_value = np.mean(arr)

    # Maximum
    mean_value = np.max(arr)

    # Range (difference between maximum and minimum)
    range_value = np.ptp(arr)

    # Variance
    variance_value = np.var(arr)

    # Median
    median_value = np.median(arr)

    # First Quartile (25th percentile)
    q1_value = np.percentile(arr, 25)

    # Third Quartile (75th percentile)
    q3_value = np.percentile(arr, 75)

    # Mean Absolute of Local Derivative
    local_derivative = np.diff(arr)  # Calculate the first difference
    mean_abs_local_derivative = np.mean(np.abs(local_derivative))

    return mean_value, mean_value, range_value, variance_value, median_value, q1_value, q3_value, mean_abs_local_derivative 

In [17]:
# Create an empty list to store the stats_df
stats_df = pd.DataFrame(columns=['speaker', 'nbr', 'classe', 'mean_f0', 'maximum_f0', 'range_f0', 'variance_f0', 'median_f0', 'first_quartile_f0', 'third_quartile_f0', 'mald_f0'
                                 ,'mean_en', 'maximum_en', 'range_en', 'variance_en', 'median_en', 'first_quartile_en', 'third_quartile_en', 'mald_en'])


for index, row in data_set.iterrows():

    speaker = row['speaker']
    nbr = row['nbr']
    classe = row['classe']
    data_f0 = row['f0']
    data_en = row['en']


    mean_f0, maximum_f0, range_f0, variance_f0, median_f0, first_quartile_f0, third_quartile_f0, mald_f0 = stats(data_f0)
    mean_en, maximum_en, range_en, variance_en, median_en, first_quartile_en, third_quartile_en, mald_en = stats(data_en)
    
    stats_df.loc[index] = [speaker, nbr, classe, mean_f0, maximum_f0, range_f0, variance_f0, median_f0, first_quartile_f0, third_quartile_f0, mald_f0,mean_en, maximum_en, range_en, variance_en, median_en, first_quartile_en, third_quartile_en, mald_en] 
    
stats_df


,speaker,nbr,classe,mean_f0,maximum_f0,range_f0,variance_f0,median_f0,first_quartile_f0,third_quartile_f0,mald_f0,mean_en,maximum_en,range_en,variance_en,median_en,first_quartile_en,third_quartile_en,mald_en
0,cy,0001,at,402.0,402.0,402.0,27176.144287,0.0,0.0,331.00,23.777778,73.0,73.0,73.0,322.208740,41.5,40.0,67.0,3.809524
1,cy,0002,at,430.0,430.0,430.0,26803.938139,0.0,0.0,251.00,15.636364,76.0,76.0,76.0,266.088625,41.0,40.0,62.0,3.272727
2,cy,0005,at,402.0,402.0,402.0,27176.144287,0.0,0.0,331.00,23.777778,73.0,73.0,73.0,322.208740,41.5,40.0,67.0,3.809524
3,cy,0006,at,430.0,430.0,430.0,26803.938139,0.0,0.0,251.00,15.636364,76.0,76.0,76.0,266.088625,41.0,40.0,62.0,3.272727
4,cy,0007,pw,257.0,257.0,257.0,10270.850538,0.0,0.0,189.50,13.683168,71.0,71.0,71.0,226.215302,52.0,41.0,66.0,2.970297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,li,1446,at,457.0,457.0,457.0,33197.267334,0.0,0.0,358.75,37.527559,80.0,80.0,80.0,301.560974,50.0,40.0,70.0,3.921260
535,li,1447,at,484.0,484.0,484.0,35521.186466,0.0,0.0,373.00,11.795455,85.0,85.0,85.0,409.966166,44.0,42.0,75.0,2.931818
536,li,1448,at,477.0,477.0,477.0,30870.875917,0.0,0.0,325.50,22.684211,86.0,86.0,86.0,362.858526,41.0,40.0,71.5,3.385965
537,li,1449,at,494.0,494.0,494.0,34149.376181,0.0,0.0,308.50,25.192982,83.0,83.0,83.0,384.437051,41.0,39.0,73.5,3.087719


## Isolating the voiced segments.

Given our understanding that the essential information is contained solely within the voiced segments, we have chosen to exclusively extract and utilize these segments for both training and testing purposes.

In [18]:
voiced_data_set = pd.DataFrame(columns=['speaker', 'nbr', 'classe', 'time', 'f0', 'en'])
stats_voiced = pd.DataFrame(columns=['speaker', 'nbr', 'classe', 'mean_f0', 'maximum_f0', 'range_f0', 'variance_f0', 'median_f0', 'first_quartile_f0', 'third_quartile_f0', 'mald_f0'
                                 ,'mean_en', 'maximum_en', 'range_en', 'variance_en', 'median_en', 'first_quartile_en', 'third_quartile_en', 'mald_en'])


for index, row in data_set.iterrows():

    speaker = row['speaker']
    nbr = row['nbr']
    classe = row['classe']
    data_f0 = row['f0']
    time = row['time']
    data_en = row['en']
    

    zero_indices = np.where(data_f0 == 0)[0]

    # Delete elements at the specified indices
    data_f0= np.delete(data_f0, zero_indices)
    time= np.delete(time, zero_indices)
    data_en= np.delete(data_en, zero_indices)

    if voiced_data_set.index.max() >= 0 :
        voiced_data_set.loc[voiced_data_set.index.max() + 1] = [speaker, nbr, classe, time, data_f0, data_en] 
    else :
        voiced_data_set.loc[0] = [speaker, nbr, classe, time, data_f0, data_en] 
    
    mean_f0, maximum_f0, range_f0, variance_f0, median_f0, first_quartile_f0, third_quartile_f0, mald_f0 = stats(data_f0)
    mean_en, maximum_en, range_en, variance_en, median_en, first_quartile_en, third_quartile_en, mald_en = stats(data_en)

    stats_voiced.loc[index] = [speaker, nbr, classe, mean_f0, maximum_f0, range_f0, variance_f0, median_f0, first_quartile_f0, third_quartile_f0, mald_f0,mean_en, maximum_en, range_en, variance_en, median_en, first_quartile_en, third_quartile_en, mald_en] 
    
stats_voiced


,speaker,nbr,classe,mean_f0,maximum_f0,range_f0,variance_f0,median_f0,first_quartile_f0,third_quartile_f0,mald_f0,mean_en,maximum_en,range_en,variance_en,median_en,first_quartile_en,third_quartile_en,mald_en
0,cy,0001,at,402.0,402.0,83.0,833.247500,333.0,331.75,379.25,6.263158,73.0,73.0,22.0,25.260000,71.0,67.75,72.00,2.315789
1,cy,0002,at,430.0,430.0,197.0,3881.358025,369.0,298.50,398.00,11.192308,76.0,76.0,19.0,27.212620,69.0,65.50,71.00,4.192308
2,cy,0005,at,402.0,402.0,83.0,833.247500,333.0,331.75,379.25,6.263158,73.0,73.0,22.0,25.260000,71.0,67.75,72.00,2.315789
3,cy,0006,at,430.0,430.0,197.0,3881.358025,369.0,298.50,398.00,11.192308,76.0,76.0,19.0,27.212620,69.0,65.50,71.00,4.192308
4,cy,0007,pw,257.0,257.0,90.0,661.604346,191.0,182.50,213.00,5.869565,71.0,71.0,16.0,17.397012,66.0,63.50,70.00,1.652174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,li,1446,at,457.0,457.0,225.0,4456.962963,378.5,309.50,408.00,21.981132,80.0,80.0,26.0,35.545953,70.5,67.00,75.75,3.075472
535,li,1447,at,484.0,484.0,233.0,3650.941358,375.0,372.75,397.00,11.000000,85.0,85.0,27.0,42.212191,77.0,72.75,80.00,2.314286
536,li,1448,at,477.0,477.0,213.0,3293.497143,379.0,338.50,417.50,15.117647,86.0,86.0,23.0,28.192653,76.0,73.00,79.50,2.588235
537,li,1449,at,494.0,494.0,281.0,6098.159240,405.0,332.00,439.00,23.444444,83.0,83.0,22.0,26.829803,78.0,75.00,81.00,2.666667



 ## Preprocessing the data to prepare it for training the classifier model.

In [19]:
data = stats_voiced[stats_voiced['classe'] != 'at']
data = data.reset_index()
data = data.drop(columns=['speaker','nbr','index'])

label_mapping = {'pw': 0, 'ap': 1}

# Apply the label mapping to the 'classe' column
data['classe'] = data['classe'].map(label_mapping)

Y_ks = data.pop('classe')
X_ks = data

X_ks

,mean_f0,maximum_f0,range_f0,variance_f0,median_f0,first_quartile_f0,third_quartile_f0,mald_f0,mean_en,maximum_en,range_en,variance_en,median_en,first_quartile_en,third_quartile_en,mald_en
0,257.0,257.0,90.0,661.604346,191.0,182.50,213.00,5.869565,71.0,71.0,16.0,17.397012,66.0,63.5,70.00,1.652174
1,250.0,250.0,83.0,511.500000,198.5,179.50,210.00,10.368421,70.0,70.0,70.0,230.600000,66.0,61.5,68.00,5.842105
2,243.0,243.0,77.0,430.389575,190.0,180.00,209.00,7.192308,74.0,74.0,20.0,17.259259,68.0,66.0,70.00,2.884615
3,221.0,221.0,67.0,436.234375,178.5,171.25,204.25,6.466667,77.0,77.0,25.0,47.312500,64.0,62.0,70.75,4.000000
4,230.0,230.0,66.0,307.820247,190.0,179.00,204.00,4.113636,71.0,71.0,19.0,14.383210,65.0,63.0,68.00,0.954545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,342.0,342.0,187.0,1697.884298,214.0,179.00,234.00,15.523810,75.0,75.0,23.0,34.900826,68.0,65.0,71.50,3.476190
369,257.0,257.0,168.0,2831.771626,216.0,183.00,240.00,13.000000,77.0,77.0,22.0,40.712803,70.0,64.0,73.00,4.312500
370,286.0,286.0,202.0,2524.239796,191.0,166.75,223.50,24.259259,71.0,71.0,24.0,51.704082,66.5,61.0,69.00,3.000000
371,255.0,255.0,168.0,2448.412098,226.0,190.00,233.50,11.090909,77.0,77.0,20.0,36.835539,71.0,65.0,74.50,2.818182


In [20]:
Y_ks

0      0
1      0
2      0
3      0
4      0
      ..
368    0
369    0
370    0
371    0
372    0
Name: classe, Length: 373, dtype: int64

## Partitioning the data into a training set (60%) and a test set (40%).

In [22]:
# Split the data into training (60%) and test (40%) sets
X_ks_train, X_ks_test, y_ks_train, y_ks_test = train_test_split(X_ks, Y_ks, test_size=0.4, random_state=42)

# Print the shapes of the resulting datasets
print("X_ks_train shape:", X_ks_train.shape)
print("X_ks_test shape:", X_ks_test.shape)
print("y_ks_train shape:", y_ks_train.shape)
print("y_ks_test shape:", y_ks_test.shape)

X_ks_train shape: (223, 16)
X_ks_test shape: (150, 16)
y_ks_train shape: (223,)
y_ks_test shape: (150,)


## Training the classification models.

## K-NN

In [42]:
# Create a k-NN classifier with k=3 (3 gives the best results)
k = 3
knn_classifier_ks = KNeighborsClassifier(n_neighbors=k)

# Fit the classifier on the training data
knn_classifier_ks.fit(X_ks_train, y_ks_train)

# Make predictions on the test data
y_pred_ks_knn = knn_classifier_ks.predict(X_ks_test)

# Calculate and print the accuracy of the classifier
accuracy_ks_knn = accuracy_score(y_ks_test, y_pred_ks_knn)
print("K-NN Accuracy:", accuracy_ks_knn)

K-NN Accuracy: 0.9


## SVC

In [43]:
# Create an SVM classifier with a linear kernel
svm_classifier_ks = SVC(kernel='linear')

# Fit the classifier on the training data
svm_classifier_ks.fit(X_ks_train, y_ks_train)

# Make predictions on the test data
y_pred_ks_svm = svm_classifier_ks.predict(X_ks_test)

# Calculate and print the accuracy of the classifier
accuracy_ks_svm = accuracy_score(y_ks_test, y_pred_ks_svm)
print("SVM Accuracy:", accuracy_ks_svm)

SVM Accuracy: 0.92


## XG Boost

In [45]:
# Create an XGBoost classifier
xgb_classifier = xgb.XGBClassifier()

# Fit the classifier on the training data
xgb_classifier.fit(X_ks_train, y_ks_train)

# Make predictions on the test data
y_pred_ks_xbg = xgb_classifier.predict(X_ks_test)

# Calculate and print the accuracy of the classifier
accuracy_ks_xbg = accuracy_score(y_ks_test, y_pred_ks_xbg)
print("XG-Boost Accuracy:", accuracy_ks_xbg)

XG-Boost Accuracy: 0.94


## Discussion : 
All classifiers achieved an accuracy exceeding 90% using their default hyperparameters. This deliberate choice aimed to ensure a fair comparison between the methods. Specifically, in the case of binary classification for the Kismet data, the K-NN classifier yielded a 90% accuracy, the SVM achieved 92%, and XG-Boost outperformed them all with an impressive 94% accuracy. These results underscore the considerable potential of feature-based voice classification.

# EXO 2

## Data preprocessing

In [47]:
data_ks_2 = stats_voiced.drop(columns=['speaker','nbr'])

label_mapping = {'pw': 0, 'ap': 1,'at': 2}

# Apply the label mapping to the 'classe' column
data_ks_2['classe'] = data_ks_2['classe'].map(label_mapping)

Y_ks_2 = data_ks_2.pop('classe')
X_ks_2 = data_ks_2

X_ks_2

,mean_f0,maximum_f0,range_f0,variance_f0,median_f0,first_quartile_f0,third_quartile_f0,mald_f0,mean_en,maximum_en,range_en,variance_en,median_en,first_quartile_en,third_quartile_en,mald_en
0,402.0,402.0,83.0,833.247500,333.0,331.75,379.25,6.263158,73.0,73.0,22.0,25.260000,71.0,67.75,72.00,2.315789
1,430.0,430.0,197.0,3881.358025,369.0,298.50,398.00,11.192308,76.0,76.0,19.0,27.212620,69.0,65.50,71.00,4.192308
2,402.0,402.0,83.0,833.247500,333.0,331.75,379.25,6.263158,73.0,73.0,22.0,25.260000,71.0,67.75,72.00,2.315789
3,430.0,430.0,197.0,3881.358025,369.0,298.50,398.00,11.192308,76.0,76.0,19.0,27.212620,69.0,65.50,71.00,4.192308
4,257.0,257.0,90.0,661.604346,191.0,182.50,213.00,5.869565,71.0,71.0,16.0,17.397012,66.0,63.50,70.00,1.652174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,457.0,457.0,225.0,4456.962963,378.5,309.50,408.00,21.981132,80.0,80.0,26.0,35.545953,70.5,67.00,75.75,3.075472
535,484.0,484.0,233.0,3650.941358,375.0,372.75,397.00,11.000000,85.0,85.0,27.0,42.212191,77.0,72.75,80.00,2.314286
536,477.0,477.0,213.0,3293.497143,379.0,338.50,417.50,15.117647,86.0,86.0,23.0,28.192653,76.0,73.00,79.50,2.588235
537,494.0,494.0,281.0,6098.159240,405.0,332.00,439.00,23.444444,83.0,83.0,22.0,26.829803,78.0,75.00,81.00,2.666667


In [48]:
Y_ks_2

0      2
1      2
2      2
3      2
4      0
      ..
534    2
535    2
536    2
537    2
538    2
Name: classe, Length: 539, dtype: int64

In [56]:
# Split the data into training (60%) and test (40%) sets
X_train_ks_2, X_test_ks_2, y_train_ks_2, y_test_ks_2 = train_test_split(X_ks_2, Y_ks_2, test_size=0.4, random_state=42)

# Print the shapes of the resulting datasets
print("X_train shape:", X_train_ks_2.shape)
print("X_test shape:", X_test_ks_2.shape)
print("y_train shape:", y_train_ks_2.shape)
print("y_test shape:", y_test_ks_2.shape)

X_train shape: (323, 16)
X_test shape: (216, 16)
y_train shape: (323,)
y_test shape: (216,)


Training the classifiers 

## K-NN

In [60]:
# Create a k-NN classifier with k=3
k = 3
knn_classifier_ks_2 = KNeighborsClassifier(n_neighbors=k)

# Fit the classifier on the training data
knn_classifier_ks_2.fit(X_train_ks_2, y_train_ks_2)

# Make predictions on the test data
y_pred_ks_2_knn = knn_classifier_ks_2.predict(X_test_ks_2)

# Calculate and print the accuracy of the classifier
accuracy_ks_2_knn = accuracy_score(y_test_ks_2, y_pred_ks_2_knn)
print("Accuracy kismet 3 classes K-nn:", accuracy_ks_2_knn)

Accuracy kismet 3 classes K-nn: 0.7314814814814815


## SVM

In [61]:
# Create an SVM classifier with a linear kernel
svm_classifier_ks_2 = SVC(kernel='linear')

# Fit the classifier on the training data
svm_classifier_ks_2.fit(X_train_ks_2, y_train_ks_2)

# Make predictions on the test data
y_pred_ks_2_svm = svm_classifier_ks_2.predict(X_test_ks_2)

# Calculate and print the accuracy of the classifier
accuracy_ks_2_svm = accuracy_score(y_test_ks_2, y_pred_ks_2_svm)
print("Accuracy kismet 3 classes SVM:", accuracy_ks_2_svm)

Accuracy kismet 3 classes SVM: 0.8379629629629629


## XG Boost

In [62]:
# Create an XGBoost classifier
xgb_classifier_ks_2 = xgb.XGBClassifier()

# Fit the classifier on the training data
xgb_classifier_ks_2.fit(X_train_ks_2, y_train_ks_2)

# Make predictions on the test data
y_pred_ks_2_xgb = xgb_classifier_ks_2.predict(X_test_ks_2)

# Calculate and print the accuracy of the classifier
accuracy_ks_2_xgb = accuracy_score(y_test_ks_2, y_pred_ks_2_xgb)
print("Accuracy kismet 3 classes XG Boost:", accuracy_ks_2_xgb)

Accuracy kismet 3 classes XG Boost: 0.8101851851851852


## Discussion :

In the context of this three-class classification task, it's evident that there has been a noticeable decline in classifier performance compared to the previously observed two-class classification. This drop in performance was somewhat anticipated due to the increased complexity of the problem. However, it's worth noting that the results remain acceptable, with accuracy ranging from a minimum of 77% to a maximum of 82%.

# EXO 3

## Importing and preparing the data  

In [64]:
# Define the directory path where the files are located
directory_path = 'BabyEars_data'

# Create an empty list to store the dataframes
data_set_2 = pd.DataFrame(columns=['id', 'classe', 'time', 'f0', 'en'])
i = 0
# Loop through each file in the directory
for filename in os.listdir(directory_path):

    id = filename[0:12]
    classe = filename[6:8]

    if not (np.any(data_set_2.loc[data_set_2['id'] == id ,'classe'] == classe)) :


        filename = filename[:-3]
        path_en = directory_path + '/' + filename +'.en'
        path_f0 = directory_path + '/' + filename +'.f0'

        #print(path_en)
        data_en = np.genfromtxt(path_en, delimiter=' ')
        data_f0 = np.genfromtxt(path_f0, delimiter=' ')

        time = data_en[:,0]

        data_en = data_en[:,1]
        data_f0 = data_f0[:,1]

        if data_set_2.index.max() >= 0 :
            
            data_set_2.loc[data_set_2.index.max() + 1] = [id, classe, time, data_f0, data_en] 
        else :
            data_set_2.loc[0] = [id, classe, time, data_f0, data_en] 

data_set_2

,id,classe,time,f0,en
0,01.01.ap.01.,ap,"[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 47.0, 44.0, 42.0, 41.0, 40.0, 41.0,..."
1,01.01.ap.06.,ap,"[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16...","[0.0, 0.0, 0.0, 0.0, 0.0, 342.0, 349.0, 376.0,...","[0.0, 0.0, 43.0, 42.0, 45.0, 60.0, 67.0, 68.0,..."
2,01.01.ap.07.,ap,"[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 45.0, 46.0, 44.0, 43.0, 43.0, 43.0, 42.0..."
3,01.01.ap.14.,ap,"[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16...","[395.0, 446.0, 442.0, 429.0, 416.0, 401.0, 405...","[0.0, 0.0, 63.0, 60.0, 58.0, 56.0, 55.0, 61.0,..."
4,01.01.ap.15.,ap,"[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 46.0, 41.0, 42.0, 43.0, 42.0, 42.0,..."
...,...,...,...,...,...
504,12.03.pr.40.,pr,"[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 101.0, 100.0, 1...","[0.0, 0.0, 16.0, 16.0, 28.0, 44.0, 49.0, 53.0,..."
505,12.03.pr.41.,pr,"[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16...","[0.0, 0.0, 0.0, 0.0, 115.0, 113.0, 115.0, 115....","[0.0, 0.0, 29.0, 42.0, 50.0, 55.0, 58.0, 61.0,..."
506,12.03.pr.44.,pr,"[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 33.0, 30.0, 30.0, 32.0, 30.0, 28.0, 24.0..."
507,12.03.pr.46.,pr,"[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16...","[0.0, 0.0, 85.0, 91.0, 93.0, 90.0, 88.0, 85.0,...","[0.0, 0.0, 47.0, 50.0, 50.0, 49.0, 48.0, 48.0,..."


In [65]:
voiced_data_set_2 = pd.DataFrame(columns=['id', 'classe', 'time', 'f0', 'en'])
stats_voiced_2 = pd.DataFrame(columns=['id', 'classe', 'mean_f0', 'maximum_f0', 'range_f0', 'variance_f0', 'median_f0', 'first_quartile_f0', 'third_quartile_f0', 'mald_f0'
                                 ,'mean_en', 'maximum_en', 'range_en', 'variance_en', 'median_en', 'first_quartile_en', 'third_quartile_en', 'mald_en'])


for index, row in data_set_2.iterrows():

    id = row['id']
    classe = row['classe']
    data_f0 = row['f0']
    time = row['time']
    data_en = row['en']
    

    zero_indices = np.where(data_f0 == 0)[0]

    # Delete elements at the specified indices
    data_f0= np.delete(data_f0, zero_indices)
    time= np.delete(time, zero_indices)
    data_en= np.delete(data_en, zero_indices)

    voiced_data_set_2.loc[index] = [id, classe, time, data_f0, data_en] 
   
    mean_f0, maximum_f0, range_f0, variance_f0, median_f0, first_quartile_f0, third_quartile_f0, mald_f0 = stats(data_f0)
    mean_en, maximum_en, range_en, variance_en, median_en, first_quartile_en, third_quartile_en, mald_en = stats(data_en)

    stats_voiced_2.loc[index] = [id, classe, mean_f0, maximum_f0, range_f0, variance_f0, median_f0, first_quartile_f0, third_quartile_f0, mald_f0,mean_en, maximum_en, range_en, variance_en, median_en, first_quartile_en, third_quartile_en, mald_en] 
    
stats_voiced_2


,id,classe,mean_f0,maximum_f0,range_f0,variance_f0,median_f0,first_quartile_f0,third_quartile_f0,mald_f0,mean_en,maximum_en,range_en,variance_en,median_en,first_quartile_en,third_quartile_en,mald_en
0,01.01.ap.01.,ap,409.0,409.0,329.0,9328.139648,201.0,159.75,289.75,29.806452,67.0,67.0,23.0,49.795898,60.5,53.25,63.25,2.032258
1,01.01.ap.06.,ap,589.0,589.0,409.0,10118.458912,258.5,223.25,325.75,9.520548,76.0,76.0,25.0,13.404675,67.0,65.25,68.00,1.164384
2,01.01.ap.07.,ap,591.0,591.0,390.0,9404.082899,261.0,216.25,325.25,13.468085,81.0,81.0,26.0,18.868056,67.0,65.00,68.00,1.680851
3,01.01.ap.14.,ap,446.0,446.0,113.0,367.996327,386.0,374.00,393.50,5.046154,78.0,78.0,78.0,164.238751,67.0,61.00,68.00,1.953846
4,01.01.ap.15.,ap,445.0,445.0,183.0,2578.106520,345.0,317.00,399.00,21.250000,70.0,70.0,19.0,21.605142,65.0,61.00,69.00,2.437500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,12.03.pr.40.,pr,128.0,128.0,28.0,116.801020,106.5,100.25,120.75,2.230769,57.0,57.0,13.0,11.658163,55.0,53.25,55.75,1.923077
505,12.03.pr.41.,pr,147.0,147.0,34.0,89.183594,121.0,118.75,126.00,2.533333,65.0,65.0,20.0,32.609375,61.5,57.25,64.00,2.333333
506,12.03.pr.44.,pr,129.0,129.0,35.0,78.066327,106.5,106.00,108.00,3.153846,57.0,57.0,7.0,4.265306,55.5,55.00,56.00,1.384615
507,12.03.pr.46.,pr,140.0,140.0,60.0,301.148789,91.0,86.00,106.00,6.125000,50.0,50.0,19.0,24.996540,48.0,47.00,49.00,1.875000


In [66]:
data_be = stats_voiced_2.drop(columns=['id'])

label_mapping = {'pr': 0, 'ap': 1,'at': 2}

# Apply the label mapping to the 'classe' column
data_be['classe'] = data_be['classe'].map(label_mapping)

Y_be = data_be.pop('classe')
X_be = data_be

X_be

,mean_f0,maximum_f0,range_f0,variance_f0,median_f0,first_quartile_f0,third_quartile_f0,mald_f0,mean_en,maximum_en,range_en,variance_en,median_en,first_quartile_en,third_quartile_en,mald_en
0,409.0,409.0,329.0,9328.139648,201.0,159.75,289.75,29.806452,67.0,67.0,23.0,49.795898,60.5,53.25,63.25,2.032258
1,589.0,589.0,409.0,10118.458912,258.5,223.25,325.75,9.520548,76.0,76.0,25.0,13.404675,67.0,65.25,68.00,1.164384
2,591.0,591.0,390.0,9404.082899,261.0,216.25,325.25,13.468085,81.0,81.0,26.0,18.868056,67.0,65.00,68.00,1.680851
3,446.0,446.0,113.0,367.996327,386.0,374.00,393.50,5.046154,78.0,78.0,78.0,164.238751,67.0,61.00,68.00,1.953846
4,445.0,445.0,183.0,2578.106520,345.0,317.00,399.00,21.250000,70.0,70.0,19.0,21.605142,65.0,61.00,69.00,2.437500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,128.0,128.0,28.0,116.801020,106.5,100.25,120.75,2.230769,57.0,57.0,13.0,11.658163,55.0,53.25,55.75,1.923077
505,147.0,147.0,34.0,89.183594,121.0,118.75,126.00,2.533333,65.0,65.0,20.0,32.609375,61.5,57.25,64.00,2.333333
506,129.0,129.0,35.0,78.066327,106.5,106.00,108.00,3.153846,57.0,57.0,7.0,4.265306,55.5,55.00,56.00,1.384615
507,140.0,140.0,60.0,301.148789,91.0,86.00,106.00,6.125000,50.0,50.0,19.0,24.996540,48.0,47.00,49.00,1.875000


In [67]:
Y_be


0      1
1      1
2      1
3      1
4      1
      ..
504    0
505    0
506    0
507    0
508    0
Name: classe, Length: 509, dtype: int64

In [68]:
# Split the data into training (60%) and test (40%) sets
X_train_be, X_test_be, y_train_be, y_test_be = train_test_split(X_be, Y_be, test_size=0.4, random_state=42)

# Print the shapes of the resulting datasets
print("X_train shape:", X_train_be.shape)
print("X_test shape:", X_test_be.shape)
print("y_train shape:", y_train_be.shape)
print("y_test shape:", y_test_be.shape)

X_train shape: (305, 16)
X_test shape: (204, 16)
y_train shape: (305,)
y_test shape: (204,)


## K-NN

In [85]:
# Create a k-NN classifier with k=5
k = 5
knn_classifier_be = KNeighborsClassifier(n_neighbors=k)

# Fit the classifier on the training data
knn_classifier_be.fit(X_train_be, y_train_be)

# Make predictions on the test data
y_pred_be = knn_classifier_be.predict(X_test_be)

# Calculate and print the accuracy of the classifier
accuracy_be = accuracy_score(y_test_be, y_pred_be)
print("Accuracy:", accuracy_be)

Accuracy: 0.46568627450980393


## Discussion :

This test highlights the increased difficulty of classifying voice segments intended for infants compared to other scenarios. This is evident in the relatively low accuracy of 47%, indicating that more than half of the classifications will be incorrect. This underscores the challenge of extracting meaningful features from infant-directed voice, as communicating with infants often involves a significant reliance on gestures and sign language rather than vocalizations. Thus, this can help explain the poorer results observed in this particular case.

## Intra




In [86]:
print("BabyEars data Accuracy whith BabyEars K-NN classifier:", accuracy_be)
print("Kismet data Accuracy whith Kismet K-NN classifier:", accuracy_ks_2_knn)

BabyEars data Accuracy whith BabyEars K-NN classifier: 0.46568627450980393
Kismet data Accuracy whith Kismet K-NN classifier: 0.7314814814814815


## Cross-corpus


In [88]:
# Make predictions on the test data
y12_pred = knn_classifier_ks_2.predict(X_test_be)
accuracy12 = accuracy_score(y_test_be, y12_pred)

# Make predictions on the test data
y21_pred = knn_classifier_be.predict(X_test_ks_2)
accuracy21 = accuracy_score(y_test_ks_2, y21_pred)

print("BabyEars data Accuracy whith Kismet K-NN classifier:", accuracy21)
print("Kismet data Accuracy whith BabyEars K-NN classifier:", accuracy12)

BabyEars data Accuracy whith Kismet K-NN classifier: 0.5462962962962963
Kismet data Accuracy whith BabyEars K-NN classifier: 0.39705882352941174


## Pooling

In [90]:
data_Kismet = stats_voiced.drop(columns=['nbr','speaker'])
data_BabyEars = stats_voiced_2.drop(columns=['id'])

concatenated_df = pd.concat([data_Kismet, data_Kismet], ignore_index=True)
concatenated_df

,classe,mean_f0,maximum_f0,range_f0,variance_f0,median_f0,first_quartile_f0,third_quartile_f0,mald_f0,mean_en,maximum_en,range_en,variance_en,median_en,first_quartile_en,third_quartile_en,mald_en
0,at,402.0,402.0,83.0,833.247500,333.0,331.75,379.25,6.263158,73.0,73.0,22.0,25.260000,71.0,67.75,72.00,2.315789
1,at,430.0,430.0,197.0,3881.358025,369.0,298.50,398.00,11.192308,76.0,76.0,19.0,27.212620,69.0,65.50,71.00,4.192308
2,at,402.0,402.0,83.0,833.247500,333.0,331.75,379.25,6.263158,73.0,73.0,22.0,25.260000,71.0,67.75,72.00,2.315789
3,at,430.0,430.0,197.0,3881.358025,369.0,298.50,398.00,11.192308,76.0,76.0,19.0,27.212620,69.0,65.50,71.00,4.192308
4,pw,257.0,257.0,90.0,661.604346,191.0,182.50,213.00,5.869565,71.0,71.0,16.0,17.397012,66.0,63.50,70.00,1.652174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,at,457.0,457.0,225.0,4456.962963,378.5,309.50,408.00,21.981132,80.0,80.0,26.0,35.545953,70.5,67.00,75.75,3.075472
1074,at,484.0,484.0,233.0,3650.941358,375.0,372.75,397.00,11.000000,85.0,85.0,27.0,42.212191,77.0,72.75,80.00,2.314286
1075,at,477.0,477.0,213.0,3293.497143,379.0,338.50,417.50,15.117647,86.0,86.0,23.0,28.192653,76.0,73.00,79.50,2.588235
1076,at,494.0,494.0,281.0,6098.159240,405.0,332.00,439.00,23.444444,83.0,83.0,22.0,26.829803,78.0,75.00,81.00,2.666667


In [91]:
label_mapping = {'pr': 0, 'pw': 0, 'ap': 1,'at': 2}

# Apply the label mapping to the 'classe' column
concatenated_df['classe'] = concatenated_df['classe'].map(label_mapping)

Y = concatenated_df.pop('classe')
X = concatenated_df

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=42)

# Print the shapes of the resulting datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (646, 16)
X_test shape: (432, 16)
y_train shape: (646,)
y_test shape: (432,)


In [92]:
# Create a k-NN classifier with k=3
k = 3
knn_classifier = KNeighborsClassifier(n_neighbors=k)

# Fit the classifier on the training data
knn_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = knn_classifier.predict(X_test)

# Calculate and print the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7638888888888888


In [93]:
# Make predictions on the test data
y31_pred = knn_classifier.predict(X_test_ks_2)
accuracy31 = accuracy_score(y_test_ks_2, y31_pred)

# Make predictions on the test data
y32_pred = knn_classifier.predict(X_test_be)
accuracy32 = accuracy_score(y_test_be, y32_pred)

y3_pred = knn_classifier.predict(X_test)
accuracy3 = accuracy_score(y_test, y3_pred)

print("BabyEars data Accuracy whith Pooling K-NN classifier:", accuracy32)
print("Kismet data Accuracy whith Pooling K-NN classifier:", accuracy31)

print("mixed data Accuracy whith Pooling K-NN classifier:", accuracy3)

BabyEars data Accuracy whith Pooling K-NN classifier: 0.4215686274509804
Kismet data Accuracy whith Pooling K-NN classifier: 0.8333333333333334
mixed data Accuracy whith Pooling K-NN classifier: 0.7638888888888888


## Discussion : 


In this section, we explored different multi-corpus approaches for our classification task. These approaches include the intra-corpus, cross-corpus, and pooling strategies, each with its own set of results and implications:

- Intra-Corpus Approach: This approach, where models are trained and tested on the same type of data, has been discussed previously. It typically yields clear and understandable results since the model is evaluated within its domain of familiarity.

- Cross-Corpus Approach: As expected, the cross-corpus approach, where models are tested on data different from their training set, produced unfavorable results. This is anticipated because the model encounters a domain shift and may struggle to generalize effectively to new data types.

- Pooling Approach: The pooling approach attempts to create a generalized model by training it on data from different types. Despite some loss in accuracy compared to models trained on single data types, this approach still demonstrated decent results, with a mixed accuracy of 76%. Such results are generally acceptable, considering the complexity of dealing with multiple data types.

It's worth noting that the pooling approach represents a compromise between the specialization of intra-corpus training and the adaptability of cross-corpus training. While it may not achieve the highest accuracy within a single data type, it provides a more versatile model capable of handling a broader range of data types.

In conclusion, the choice of multi-corpus approach depends on the specific goals and constraints of the classification task. The pooling approach, in particular, strikes a balance between domain specialization and generalization, offering reasonable performance across different data types.